In [7]:
import cv2
import mediapipe as mp
import numpy as np
import tensorflow as tf
import math
mp_drawing = mp.solutions.drawing_utils
mp_pose = mp.solutions.pose

In [1]:
def calculate_dist(a,b):
    a = np.array(a) # First
    b = np.array(b) # Mid

    
    dist = np.linalg.norm(a-b)*100

    if dist<0:
        dist=-dist

    return dist

In [3]:
def calculate_angle(a,b,c):
    a = np.array(a) # First
    b = np.array(b) # Mid
    c = np.array(c) # End
    
    ang = math.degrees(math.atan2(c[1]-b[1], c[0]-b[0]) - math.atan2(a[1]-b[1], a[0]-b[0]))
    if ang<0 :
      ang=ang+360
    if ang >= 360- ang:
        ang=360-ang
    return ang

In [4]:
#HighV
def HighV(hand,elbow,shoulder,foot,knee):
    score=0
    a=['','','','','']
    hand=np.array(hand)
    elbow=np.array(elbow)
    shoulder=np.array(shoulder)
    knee=np.array(knee)
    #上半部
    if shoulder>110 and shoulder<=140:  #手臂開闔程度
        t3=20
        a[2]='OK'
    else:
        t3=0
        a[2]=''
    if hand>=172: #手腕彎曲
        t1=5
        a[0]='OK'
    else:
        t1=0
        a[0]=''
    if elbow>=170: #手臂彎曲
        t2=10
        a[1]='OK'
    else:
        t2=0
        a[1]=''
    #下半部
    if foot<=95 and foot>80: #腳開闔程度
        d1=10
        a[3]='OK'
    else:
        d1=0
        a[3]='' 
    if knee<=180 and knee>170: #腳
        d2=5
        a[4]='OK'
    else:
        d2=0
        a[4]=''        
    
    score=t1+t2+t3+d1+d2
    
    return a,score

In [70]:
#Right Punch
#右邊
def R_Punch(hand,elbow,shoulder,foot,knee):
    score=0
    a=[]
    hand=np.array(hand)
    elbow=np.array(elbow)
    shoulder=np.array(shoulder)
    knee=np.array(knee)
    #上半部
    if shoulder>108 and shoulder<=120: #手臂開闔程度
        t3=20   
        if "arms too open" in a:
            a.remove("arms too open")
    else:
        t3=0
        a.append("arms too open")
    if hand>=140: #手腕彎曲
        t1=5 
        if "wrist bend" in a:
            a.remove("wrist bend")

    else:
        t1=0
        a.append("wrist bend")
    
        
    if elbow>=165: #手臂彎曲 arm bent
        t2=10  
        if "arm bent" in a:
            a.remove("arm bent")
    else:
        t2=0
        a.append("arm bent")
    #下半部
    if foot<=100 and foot>80:#腳開闔程度feet too wide
        d1=10
        if "feet too wide" in a:
            a.remove("feet too wide")
    else:
        d1=0
        a.append("feet too wide")
    if knee<=180 and knee>170: #腳歪曲crooked feet
        d2=5
        if "crooked feet" in a:
            a.remove("crooked feet")
    else:
        d2=0
        a.append("crooked feet")        
    
    score=t1+t2+t3+d1+d2
    
    return a,score
#左邊
def L_Punch(hand,elbow,shoulder,foot,knee):
    score=0
    a=[]
    hand=np.array(hand)
    elbow=np.array(elbow)
    shoulder=np.array(shoulder)
    knee=np.array(knee)
    #上半部
    if shoulder>120 and shoulder<=160: #手臂開闔程度
        t3=20   
        if "arms too open" in a:
            a.remove("arms too open")
    else:
        t3=0
        a.append("arms too open")
    if hand>140 and hand<180: #手腕彎曲
        t1=5 
        if "wrist bend" in a:
            a.remove("wrist bend")

    else:
        t1=0
        a.append("wrist bend")
    
        
    if elbow>=50 and elbow<70: #手臂彎曲 arm bent
        t2=10  
        if "arm bent" in a:
            a.remove("arm bent")
    else:
        t2=0
        a.append("arm bent")
    #下半部
    if foot<=4 :#腳開闔程度feet too wide
        d1=10
        if "feet too wide" in a:
            a.remove("feet too wide")
    else:
        d1=0
        a.append("feet too wide")
    if knee<=180 and knee>170: #腳歪曲crooked feet
        d2=5
        if "crooked feet" in a:
            a.remove("crooked feet")
    else:
        d2=0
        a.append("crooked feet")        
    
    score=t1+t2+t3+d1+d2
    
    return a,score    


In [71]:
import cv2
import numpy as np

file_name = "G:\Score\Punch\\Punch100_1.jpg"  #图片的路径
#cv2.namedWindow('image',cv2.WINDOW_NORMAL)  #创建窗口
#cv2.resizeWindow("image", 1200, 800)  #调整窗口大小
#cap = cv2.VideoCapture('022.mp4')
s21=s22=s12=s11=0
score=100
# Curl counter variables
counter = 0 
stagea=None
stage = []
stage1 = []
n=0

## Setup mediapipe instance
with mp_pose.Pose(min_detection_confidence=0.5, min_tracking_confidence=0.5) as pose:
    
    ret=cv2.imread(file_name, 1)
    frame = cv2.imread(file_name, 1)  #cv2.imread(文件名，标记)读入图像

    #frame=cv2.flip(frame,-1) #翻轉：-1上下、0上下左右、1左右
        # Recolor image to RGB
        
        
    image = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    image.flags.writeable = False
   
        #cv2.resizeWindow("image", 1200, 900)    # 設定長和寬
        # Make detection
    results = pose.process(image)
    
        # Recolor back to BGR
    image.flags.writeable = True
    image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)
       
        # Extract landmarks
    try:
        landmarks = results.pose_landmarks.landmark
            

            # Get coordinates
            #上半部
        left_shoulder = [landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER.value].x,landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER.value].y]
        right_shoulder = [landmarks[mp_pose.PoseLandmark.RIGHT_SHOULDER.value].x,landmarks[mp_pose.PoseLandmark.RIGHT_SHOULDER.value].y]
        left_index = [landmarks[mp_pose.PoseLandmark.LEFT_INDEX.value].x,landmarks[mp_pose.PoseLandmark.LEFT_INDEX.value].y]
        right_index = [landmarks[mp_pose.PoseLandmark.RIGHT_INDEX.value].x,landmarks[mp_pose.PoseLandmark.RIGHT_INDEX.value].y]
        left_elbow = [landmarks[mp_pose.PoseLandmark.LEFT_ELBOW.value].x,landmarks[mp_pose.PoseLandmark.LEFT_ELBOW.value].y]
        left_wrist = [landmarks[mp_pose.PoseLandmark.LEFT_WRIST.value].x,landmarks[mp_pose.PoseLandmark.LEFT_WRIST.value].y]
        right_elbow = [landmarks[mp_pose.PoseLandmark.RIGHT_ELBOW.value].x,landmarks[mp_pose.PoseLandmark.RIGHT_ELBOW.value].y]
        right_wrist = [landmarks[mp_pose.PoseLandmark.RIGHT_WRIST.value].x,landmarks[mp_pose.PoseLandmark.RIGHT_WRIST.value].y]
            
            
            #下半部
        left_hip = [landmarks[mp_pose.PoseLandmark.LEFT_HIP.value].x,landmarks[mp_pose.PoseLandmark.LEFT_HIP.value].y]
        right_hip = [landmarks[mp_pose.PoseLandmark.RIGHT_HIP.value].x,landmarks[mp_pose.PoseLandmark.RIGHT_HIP.value].y]
        left_knee = [landmarks[mp_pose.PoseLandmark.LEFT_KNEE.value].x,landmarks[mp_pose.PoseLandmark.LEFT_KNEE.value].y]
        left_ankle = [landmarks[mp_pose.PoseLandmark.LEFT_ANKLE.value].x,landmarks[mp_pose.PoseLandmark.LEFT_ANKLE.value].y]
        right_knee = [landmarks[mp_pose.PoseLandmark.RIGHT_KNEE.value].x,landmarks[mp_pose.PoseLandmark.RIGHT_KNEE.value].y]
        right_ankle = [landmarks[mp_pose.PoseLandmark.RIGHT_ANKLE.value].x,landmarks[mp_pose.PoseLandmark.RIGHT_ANKLE.value].y]
            
            # Calculate angle
            #right_angle = calculate_angle( left_shoulder,right_shoulder,right_wrist )
                        
            # Visualize angle
            
        cv2.putText(image, str(int(calculate_angle(right_elbow,right_wrist,right_index))), 
                           tuple(np.multiply(right_index, [640, 480]).astype(int)), 
                           cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 205), 2, cv2.LINE_AA
                                )
                                
        cv2.putText(image, str(int(calculate_angle(left_index,left_wrist,left_elbow))), 
                           tuple(np.multiply(left_index, [640, 480]).astype(int)), 
                           cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0,250, 205), 2, cv2.LINE_AA
                                )            
        cv2.putText(image, str(int(calculate_angle(right_shoulder,right_elbow,right_wrist))), 
                           tuple(np.multiply(right_elbow, [640, 480]).astype(int)), 
                           cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 205), 2, cv2.LINE_AA
                                )
                                
        cv2.putText(image, 'LE'+str(int(calculate_angle(left_wrist,left_elbow,left_shoulder))), 
                           tuple(np.multiply(left_elbow, [640, 480]).astype(int)), 
                           cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0,250, 205), 2, cv2.LINE_AA
                                )
        cv2.putText(image, str(int(calculate_angle( left_shoulder,right_shoulder,right_elbow))), 
                           tuple(np.multiply(right_shoulder, [640, 480]).astype(int)), 
                           cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 205), 2, cv2.LINE_AA
                                )
                                
        cv2.putText(image, 'LS'+str(int(calculate_angle( left_elbow,left_shoulder,right_shoulder))), 
                           tuple(np.multiply(left_shoulder, [640, 480]).astype(int)), 
                           cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0,250, 205), 2, cv2.LINE_AA
                                )
                                
        cv2.putText(image, str(int(calculate_angle( right_hip,right_knee,right_ankle))), 
                           tuple(np.multiply(right_knee, [640, 480]).astype(int)), 
                           cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 205), 2, cv2.LINE_AA
                                )
                                
        cv2.putText(image, 'L'+str(int(calculate_angle( left_hip,left_knee,left_ankle))), 
                           tuple(np.multiply(left_knee, [640, 480]).astype(int)), 
                           cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0,250, 205), 2, cv2.LINE_AA                                
                                )
            
        cv2.putText(image, str(int(calculate_dist(right_ankle,left_ankle))), 
                           tuple(np.multiply(right_ankle, [640, 480]).astype(int)), 
                           cv2.FONT_HERSHEY_SIMPLEX, 0.5, (203,220, 0), 2, cv2.LINE_AA                                
                                )
            
            # Curl counter logic
           
                #上半部檢測
        stage1,s11=R_Punch(calculate_angle(right_elbow,right_wrist,right_index),
            calculate_angle(right_shoulder,right_elbow,right_wrist),
            calculate_angle( left_shoulder,right_shoulder,right_elbow),
            calculate_angle( right_knee,right_hip,left_hip),
            calculate_angle( right_hip,right_knee,right_ankle)
            )            
        stage,s21=L_Punch(calculate_angle(left_index,left_wrist,left_elbow),
            calculate_angle(left_wrist,left_elbow,left_shoulder),
            calculate_angle( left_elbow,left_shoulder,right_shoulder),
            calculate_dist(right_ankle,left_ankle),
            calculate_angle( left_hip,left_knee,left_ankle),
            )
        score= s11+s21
            
    except:
        pass
        #cv2.rectangle(image, (0,0), (350,230), (0,0,0), -1)
    cv2.putText(image, str(score), 
                    (170,29), 
                    cv2.FONT_HERSHEY_SIMPLEX, 1, (255,255,255), 2, cv2.LINE_AA)
    cv2.putText(image, "R"+str(stage1), 
                    (10,150), 
                    cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255,255,255), 2, cv2.LINE_AA)
    cv2.putText(image, "L"+str(stage), 
                    (10,60), 
                    cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255,255,255), 2, cv2.LINE_AA)
        
        # Render detections
   
    mp_drawing.draw_landmarks(image, results.pose_landmarks, mp_pose.POSE_CONNECTIONS,
                                mp_drawing.DrawingSpec(color=(205,117,66), thickness=2, circle_radius=1), 
                                mp_drawing.DrawingSpec(color=(245,66,230), thickness=2, circle_radius=2) 
                                 )               
        #image=cv2.flip(image,1) #翻轉：-1上下、0上下左右、1左右
    cv2.imshow('image', image)
    cv2.waitKey(500000000)  
    
    cv2.destroyAllWindows()  
        
    
   
        
    




In [58]:
cv2.destroyAllWindows()

In [ ]:
cap = cv2.VideoCapture(0)
#cap = cv2.VideoCapture('022.mp4')
s21=s22=s12=s11=0
score=100
# Curl counter variables
counter = 0 
stagea=None
stage = []
stage1 = []
n=0

## Setup mediapipe instance
with mp_pose.Pose(min_detection_confidence=0.5, min_tracking_confidence=0.5) as pose:
    while cap.isOpened():
        ret, frame = cap.read()
        frame=cv2.flip(frame,1) #翻轉：-1上下、0上下左右、1左右
        # Recolor image to RGB
        
        
        image = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        image.flags.writeable = False
        cv2.namedWindow("image", 0)  # 0可調大小，注意：視窗名必須imshow裡面的一視窗名一直
        #cv2.resizeWindow("image", 1200, 900)    # 設定長和寬
        # Make detection
        results = pose.process(image)
    
        # Recolor back to BGR
        image.flags.writeable = True
        image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)
       
        # Extract landmarks
        try:
            landmarks = results.pose_landmarks.landmark
            

            # Get coordinates
            #上半部
            left_shoulder = [landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER.value].x,landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER.value].y]
            right_shoulder = [landmarks[mp_pose.PoseLandmark.RIGHT_SHOULDER.value].x,landmarks[mp_pose.PoseLandmark.RIGHT_SHOULDER.value].y]
            left_index = [landmarks[mp_pose.PoseLandmark.LEFT_INDEX.value].x,landmarks[mp_pose.PoseLandmark.LEFT_INDEX.value].y]
            right_index = [landmarks[mp_pose.PoseLandmark.RIGHT_INDEX.value].x,landmarks[mp_pose.PoseLandmark.RIGHT_INDEX.value].y]
            left_elbow = [landmarks[mp_pose.PoseLandmark.LEFT_ELBOW.value].x,landmarks[mp_pose.PoseLandmark.LEFT_ELBOW.value].y]
            left_wrist = [landmarks[mp_pose.PoseLandmark.LEFT_WRIST.value].x,landmarks[mp_pose.PoseLandmark.LEFT_WRIST.value].y]
            right_elbow = [landmarks[mp_pose.PoseLandmark.RIGHT_ELBOW.value].x,landmarks[mp_pose.PoseLandmark.RIGHT_ELBOW.value].y]
            right_wrist = [landmarks[mp_pose.PoseLandmark.RIGHT_WRIST.value].x,landmarks[mp_pose.PoseLandmark.RIGHT_WRIST.value].y]
            
            
            #下半部
            left_hip = [landmarks[mp_pose.PoseLandmark.LEFT_HIP.value].x,landmarks[mp_pose.PoseLandmark.LEFT_HIP.value].y]
            right_hip = [landmarks[mp_pose.PoseLandmark.RIGHT_HIP.value].x,landmarks[mp_pose.PoseLandmark.RIGHT_HIP.value].y]
            left_knee = [landmarks[mp_pose.PoseLandmark.LEFT_KNEE.value].x,landmarks[mp_pose.PoseLandmark.LEFT_KNEE.value].y]
            left_ankle = [landmarks[mp_pose.PoseLandmark.LEFT_ANKLE.value].x,landmarks[mp_pose.PoseLandmark.LEFT_ANKLE.value].y]
            right_knee = [landmarks[mp_pose.PoseLandmark.RIGHT_KNEE.value].x,landmarks[mp_pose.PoseLandmark.RIGHT_KNEE.value].y]
            right_ankle = [landmarks[mp_pose.PoseLandmark.RIGHT_ANKLE.value].x,landmarks[mp_pose.PoseLandmark.RIGHT_ANKLE.value].y]
            
            # Calculate angle
            #right_angle = calculate_angle( left_shoulder,right_shoulder,right_wrist )
                        
            # Visualize angle
            
            cv2.putText(image, str(int(calculate_angle(right_elbow,right_wrist,right_index))), 
                           tuple(np.multiply(right_index, [640, 480]).astype(int)), 
                           cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 205), 2, cv2.LINE_AA
                                )
                                
            cv2.putText(image, str(int(calculate_angle(left_index,left_wrist,left_elbow))), 
                           tuple(np.multiply(left_index, [640, 480]).astype(int)), 
                           cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0,250, 205), 2, cv2.LINE_AA
                                )            
            cv2.putText(image, str(int(calculate_angle(right_shoulder,right_elbow,right_wrist))), 
                           tuple(np.multiply(right_elbow, [640, 480]).astype(int)), 
                           cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 205), 2, cv2.LINE_AA
                                )
                                
            cv2.putText(image, 'LE'+str(int(calculate_angle(left_wrist,left_elbow,left_shoulder))), 
                           tuple(np.multiply(left_elbow, [640, 480]).astype(int)), 
                           cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0,250, 205), 2, cv2.LINE_AA
                                )
            cv2.putText(image, str(int(calculate_angle( left_shoulder,right_shoulder,right_elbow))), 
                           tuple(np.multiply(right_shoulder, [640, 480]).astype(int)), 
                           cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 205), 2, cv2.LINE_AA
                                )
                                
            cv2.putText(image, 'LS'+str(int(calculate_angle( left_elbow,left_shoulder,right_shoulder))), 
                           tuple(np.multiply(left_shoulder, [640, 480]).astype(int)), 
                           cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0,250, 205), 2, cv2.LINE_AA
                                )
                                
            cv2.putText(image, str(int(calculate_angle( right_hip,right_knee,right_ankle))), 
                           tuple(np.multiply(right_knee, [640, 480]).astype(int)), 
                           cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 205), 2, cv2.LINE_AA
                                )
                                
            cv2.putText(image, 'L'+str(int(calculate_angle( left_hip,left_knee,left_ankle))), 
                           tuple(np.multiply(left_knee, [640, 480]).astype(int)), 
                           cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0,250, 205), 2, cv2.LINE_AA                                
                                )
            
            cv2.putText(image, str(int(calculate_dist(right_ankle,left_ankle))), 
                           tuple(np.multiply(right_ankle, [640, 480]).astype(int)), 
                           cv2.FONT_HERSHEY_SIMPLEX, 0.5, (203,220, 0), 2, cv2.LINE_AA                                
                                )
            
            # Curl counter logic
           
                #上半部檢測
            stage1,s11=R_Punch(calculate_angle(right_elbow,right_wrist,right_index),
            calculate_angle(right_shoulder,right_elbow,right_wrist),
            calculate_angle( left_shoulder,right_shoulder,right_elbow),
            calculate_angle( right_knee,right_hip,left_hip),
            calculate_angle( right_hip,right_knee,right_ankle)
            )            
            stage,s21=L_Punch(calculate_angle(left_index,left_wrist,left_elbow),
            calculate_angle(left_wrist,left_elbow,left_shoulder),
            calculate_angle( left_elbow,left_shoulder,right_shoulder),
            calculate_dist(right_ankle,left_ankle),
            calculate_angle( left_hip,left_knee,left_ankle),
            )
            score= s11+s21
            
        except:
            pass
        #cv2.rectangle(image, (0,0), (350,230), (0,0,0), -1)
        cv2.putText(image, str(score), 
                    (170,29), 
                    cv2.FONT_HERSHEY_SIMPLEX, 1, (255,255,255), 2, cv2.LINE_AA)
        cv2.putText(image, "R"+str(stage1), 
                    (10,150), 
                    cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255,255,255), 2, cv2.LINE_AA)
        cv2.putText(image, "L"+str(stage), 
                    (10,60), 
                    cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255,255,255), 2, cv2.LINE_AA)
        
        # Render detections
   
        mp_drawing.draw_landmarks(image, results.pose_landmarks, mp_pose.POSE_CONNECTIONS,
                                mp_drawing.DrawingSpec(color=(205,117,66), thickness=2, circle_radius=1), 
                                mp_drawing.DrawingSpec(color=(245,66,230), thickness=2, circle_radius=2) 
                                 )               
        #image=cv2.flip(image,1) #翻轉：-1上下、0上下左右、1左右
        
        
        key=cv2.waitKey(10) #等候使用者按鍵盤指令
        print(calculate_dist(right_ankle,left_ankle))
        if key==ord('a'):  #a拍照
            cv2.imwrite(f'-photo-{n}.jpg',image) #拍照
            n=n+1
            fs=score
            print(fs)
            print("左邊")
            for i in range(len(stage)):
                print(stage[i])
            print("右邊")                
            for i in range(len(stage1)):
                print(stage1[i])
        if key==ord('q'):  #q退出
            break
        cv2.imshow('image', image)
        

    cap.release()
    cv2.destroyAllWindows()

In [21]:
cap.release()
cv2.destroyAllWindows()

AttributeError: 'numpy.ndarray' object has no attribute 'release'

In [ ]:
cap = cv2.VideoCapture(0)
#cap = cv2.VideoCapture('022.mp4')

# Curl counter variables
counter = 0 
stage = ["RShoulder","LShoulder"]
stage1 = ["RFOOT","LFOOT"]

## Setup mediapipe instance
with mp_pose.Pose(min_detection_confidence=0.5, min_tracking_confidence=0.5) as pose:
    while cap.isOpened():
        ret, frame = cap.read()
        frame=cv2.flip(frame,1) #翻轉：-1上下、0上下左右、1左右
        # Recolor image to RGB
        image = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        image.flags.writeable = False
      
        # Make detection
        results = pose.process(image)
    
        # Recolor back to BGR
        image.flags.writeable = True
        image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)
        
        # Extract landmarks
        try:
            landmarks = results.pose_landmarks.landmark
            
            # Get coordinates
            #上半部
            left_shoulder = [landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER.value].x,landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER.value].y]
            right_shoulder = [landmarks[mp_pose.PoseLandmark.RIGHT_SHOULDER.value].x,landmarks[mp_pose.PoseLandmark.RIGHT_SHOULDER.value].y]
            left_elbow = [landmarks[mp_pose.PoseLandmark.LEFT_ELBOW.value].x,landmarks[mp_pose.PoseLandmark.LEFT_ELBOW.value].y]
            left_wrist = [landmarks[mp_pose.PoseLandmark.LEFT_WRIST.value].x,landmarks[mp_pose.PoseLandmark.LEFT_WRIST.value].y]
            right_elbow = [landmarks[mp_pose.PoseLandmark.RIGHT_ELBOW.value].x,landmarks[mp_pose.PoseLandmark.RIGHT_ELBOW.value].y]
            right_wrist = [landmarks[mp_pose.PoseLandmark.RIGHT_WRIST.value].x,landmarks[mp_pose.PoseLandmark.RIGHT_WRIST.value].y]
            #下半部
            left_hip = [landmarks[mp_pose.PoseLandmark.LEFT_HIP.value].x,landmarks[mp_pose.PoseLandmark.LEFT_HIP.value].y]
            right_hip = [landmarks[mp_pose.PoseLandmark.RIGHT_HIP.value].x,landmarks[mp_pose.PoseLandmark.RIGHT_HIP.value].y]
            left_knee = [landmarks[mp_pose.PoseLandmark.LEFT_KNEE.value].x,landmarks[mp_pose.PoseLandmark.LEFT_KNEE.value].y]
            left_ankle = [landmarks[mp_pose.PoseLandmark.LEFT_ANKLE.value].x,landmarks[mp_pose.PoseLandmark.LEFT_ANKLE.value].y]
            right_knee = [landmarks[mp_pose.PoseLandmark.RIGHT_KNEE.value].x,landmarks[mp_pose.PoseLandmark.RIGHT_KNEE.value].y]
            right_ankle = [landmarks[mp_pose.PoseLandmark.RIGHT_ANKLE.value].x,landmarks[mp_pose.PoseLandmark.RIGHT_ANKLE.value].y]
            
            # Calculate angle
            #right_angle = calculate_angle( left_shoulder,right_shoulder,right_wrist )
                        
            # Visualize angle
            cv2.putText(image, str(calculate_angle(right_hip,right_knee,right_ankle)), 
                           tuple(np.multiply(right_hip, [640, 480]).astype(int)), 
                           cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 255, 255), 2, cv2.LINE_AA
                                )
            
            # Curl counter logic
            #High V
            #上半部檢測
            ls_stage=shoulder(calculate_angle(left_shoulder,right_shoulder,right_wrist ))
            rs_stage=shoulder(calculate_angle( left_wrist,left_shoulder,right_shoulder ))
            #下半部檢測
            lh_stage=hip(calculate_angle(right_shoulder,right_hip,right_ankle ),calculate_angle(right_hip,right_knee,right_ankle ))
            rh_stage=hip(calculate_angle(left_shoulder,left_hip,left_ankle ),calculate_angle(left_hip,left_knee,left_ankle ))
            stage[0]="RS"+rs_stage
            stage[1]="LS"+ls_stage
            stage1[0]="R"+rh_stage
            stage1[1]="L"+lh_stage
            
            #倒立
              
                
                       
        except:
            pass
        
        # Render curl counter
        # Setup status box
        cv2.rectangle(image, (0,0), (225,250), (245,117,16), -1)
        
        # Rep data
        cv2.putText(image, 'RS_STAGE', (15,12), 
                    cv2.FONT_HERSHEY_SIMPLEX, 0.3, (0,0,0), 1, cv2.LINE_AA)
        cv2.putText(image, str(stage), 
                    (10,60), 
                    cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255,255,255), 2, cv2.LINE_AA)
        
        # Stage data
        cv2.putText(image, 'LS_STAGE', (15,82), 
                    cv2.FONT_HERSHEY_SIMPLEX, 0.3, (0,0,0), 1, cv2.LINE_AA)
        cv2.putText(image, str(stage1), 
                    (10,150), 
                    cv2.FONT_HERSHEY_SIMPLEX, 1, (255,255,255), 2, cv2.LINE_AA)
        
        
        # Render detections
        mp_drawing.draw_landmarks(image, results.pose_landmarks, mp_pose.POSE_CONNECTIONS,
                                mp_drawing.DrawingSpec(color=(245,117,66), thickness=2, circle_radius=2), 
                                mp_drawing.DrawingSpec(color=(245,66,230), thickness=2, circle_radius=2) 
                                 )               
        
        cv2.imshow('Mediapipe Feed', image)

        if cv2.waitKey(10) & 0xFF == ord('q'):
            break

    cap.release()
    cv2.destroyAllWindows()

In [ ]:
import cv2
import mediapipe as mp
import numpy as np
import time
import json
#import serial

cam = cv2.VideoCapture(0)
mppose = mp.solutions.pose
mpdraw = mp.solutions.drawing_utils
poses = mppose.Pose()
h = 0
w = 0
#ser = serial.Serial("COM3", 9600)

start_time = 0
status = False

sport = {
    "name": "Squat",
    "count": 0,
    "calories": 0
}


def logger(count, cals):
    f = open("log.txt", 'a')
    fs = f"{time.ctime()} count: {count} cals: {cals}\n"
    f.write(fs)
    f.close()


def calc_angles(a, b, c):
    a = np.array(a)
    b = np.array(b)
    c = np.array(c)

    radians = np.arctan2(c[1] - b[1], c[0] - b[0]) - \
              np.arctan2(a[1] - b[1], a[0] - b[0])

    angle = np.abs(radians * 180.0 / np.pi)

    if angle > 180:
        angle = 360 - angle

    return angle


def get_landmark(landmarks, part_name):
    return [
        landmarks[mppose.PoseLandmark[part_name].value].x,
        landmarks[mppose.PoseLandmark[part_name].value].y,
        landmarks[mppose.PoseLandmark[part_name].value].z,
    ]


def get_visibility(landmarks):
    if landmarks[mppose.PoseLandmark["RIGHT_HIP"].value].visibility < 0.8 or \
            landmarks[mppose.PoseLandmark["LEFT_HIP"].value].visibility < 0.8:
        return False
    else:
        return True


def get_body_ratio(landmarks):
    r_body = abs(landmarks[mppose.PoseLandmark["RIGHT_SHOULDER"].value].y
                 - landmarks[mppose.PoseLandmark["RIGHT_HIP"].value].y)
    l_body = abs(landmarks[mppose.PoseLandmark["LEFT_SHOULDER"].value].y
                 - landmarks[mppose.PoseLandmark["LEFT_HIP"].value].y)
    avg_body = (r_body + l_body) / 2
    r_leg = abs(landmarks[mppose.PoseLandmark["RIGHT_HIP"].value].y
                - landmarks[mppose.PoseLandmark["RIGHT_ANKLE"].value].y)
    l_leg = abs(landmarks[mppose.PoseLandmark["LEFT_HIP"].value].y
                - landmarks[mppose.PoseLandmark["LEFT_ANKLE"].value].y)
    if r_leg > l_leg:
        return r_leg / avg_body
    else:
        return l_leg / avg_body


def get_knee_angle(landmarks):
    r_hip = get_landmark(landmarks, "RIGHT_HIP")
    l_hip = get_landmark(landmarks, "LEFT_HIP")

    r_knee = get_landmark(landmarks, "RIGHT_KNEE")
    l_knee = get_landmark(landmarks, "LEFT_KNEE")

    r_ankle = get_landmark(landmarks, "RIGHT_ANKLE")
    l_ankle = get_landmark(landmarks, "LEFT_ANKLE")

    r_angle = calc_angles(r_hip, r_knee, r_ankle)
    l_angle = calc_angles(l_hip, l_knee, l_ankle)

    m_hip = (r_hip + l_hip)
    m_hip = [x / 2 for x in m_hip]
    m_knee = (r_knee + l_knee)
    m_knee = [x / 2 for x in m_knee]
    m_ankle = (r_ankle + l_ankle)
    m_ankle = [x / 2 for x in m_ankle]

    mid_angle = calc_angles(m_hip, m_knee, m_ankle)

    return [r_angle, l_angle, mid_angle]


def main():
    global h, w, start_time, status
    flag = False
    if not cam.isOpened():
        print("Camera not open")
        exit()

    try:
        f = open("sport_recorder.json", "r")
        prevdata = json.load(f)
        if sport['name'] == prevdata['name']:
            sport['count'] = prevdata['count']
            sport['calories'] = prevdata['calories']
            print("Read Success!")
        f.close()
    except:
        print("Read Error...")
        pass

    tmp = f"a{sport['count']}\n"
    #ser.write(str.encode(tmp))
    tmp = f"b{sport['calories']}\n"
    #ser.write(str.encode(tmp))

    cv2.namedWindow('frame', cv2.WINDOW_FREERATIO)

    while not flag:
        ret, frame = cam.read()
        if not ret:
            print("Read Error")
            break
        frame = cv2.flip(frame, 1)
        rgbframe = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        poseoutput = poses.process(rgbframe)
        h, w, _ = frame.shape
        preview = frame.copy()

        if poseoutput.pose_landmarks:
            mpdraw.draw_landmarks(preview, poseoutput.pose_landmarks, mppose.POSE_CONNECTIONS)
            knee_angles = get_knee_angle(poseoutput.pose_landmarks.landmark)
            body_ratio = get_body_ratio(poseoutput.pose_landmarks.landmark)
            if knee_angles[0] < 120:
                cv2.putText(preview, "Left: Down {:.1f}".format(knee_angles[0]), (10, 40)
                            , cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 1, cv2.LINE_AA
                            )
            elif knee_angles[0] < 130:
                cv2.putText(preview, "Left: ??? {:.1f}".format(knee_angles[0]), (10, 40)
                            , cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 255), 1, cv2.LINE_AA
                            )
            else:
                cv2.putText(preview, "Left: Up {:.1f}".format(knee_angles[0]), (10, 40)
                            , cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 255), 1, cv2.LINE_AA
                            )

            if knee_angles[1] < 120:
                cv2.putText(preview, "Right: Down {:.1f}".format(knee_angles[1]), (10, 80)
                            , cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 1, cv2.LINE_AA
                            )
            elif knee_angles[1] < 130:
                cv2.putText(preview, "Right: ??? {:.1f}".format(knee_angles[1]), (10, 80)
                            , cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 255), 1, cv2.LINE_AA
                            )
            else:
                cv2.putText(preview, "Right: Up {:.1f}".format(knee_angles[1]), (10, 80)
                            , cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 255), 1, cv2.LINE_AA
                            )

            avg_angle = (knee_angles[0] + knee_angles[1]) // 2

            # determine the status
            if status:
                if avg_angle > 160:
                    status = False
                    pass_time = time.time() - start_time
                    start_time = 0
                    if 3000 > pass_time > 3:
                        sport['count'] = sport['count'] + 1
                        sport['calories'] = sport['calories'] + int(0.66 * pass_time)
                        logger(sport['count'], sport['calories'])
                        tmp = f"a{sport['count']}\n"
                        #ser.write(str.encode(tmp))
                        tmp = f"b{sport['calories']}\n"
                        #ser.write(str.encode(tmp))

            else:
                if avg_angle < 120 and body_ratio < 1.2:
                    start_time = time.time()
                    status = True

            # print(f"status:{status} {start_time}")
            if status:
                cv2.putText(preview, f"{status} : {avg_angle:.1f} {body_ratio:.3f}", (10, 120)
                            , cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 1, cv2.LINE_AA
                            )
                #if time.time() - start_time > 3:
                    #ser.write(b'command_2\n')
                #else:
                    #ser.write(b'command_1\n')
            else:
                cv2.putText(preview, f"{status} : {avg_angle:.1f} {body_ratio:.3f}", (10, 120)
                            , cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 255), 1, cv2.LINE_AA
                            )
                #ser.write(b'command_4\n')
        else:
            #ser.write(b'command_4\n')
            start_time = 0

        cv2.imshow('frame', preview)
        if cv2.waitKey(1) & 0xFF == ord('q'):
            flag = True

    f = open("sport_recorder.json", "w+")
    f.write(json.dumps(sport))
    f.close()

    # release camera
    cam.release()
    cv2.destroyAllWindows()


if __name__ == '__main__':
    main()